In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
from scipy.optimize import fsolve
from scipy.optimize import root

np.set_printoptions(threshold=10000)
plt.rcParams['text.usetex'] = True
# plt.rcParams['font.sans-serif'] = 'Times New Roman'
# plt.rcParams['mathtext.fontset'] = 'cm'

In [2]:
# 导入之前计算的参数与历年均衡相对变化
param = scio.loadmat('./part0_param.mat')
initvalue = scio.loadmat('./part0_initvalue.mat')
data = {**param, **initvalue}

for var_name in data: 
    if not var_name.startswith('__'): 
        exec("%s = np.asarray(data['%s'])" % (var_name, var_name))

In [3]:
# 初始化：相当于重新求解了一次2012年的均衡
year = 2012
N0 = N0_02
N = N_0212[:, year-2002:year-2001]
R = R_0212[:, year-2002:year-2001]
I = I_0212[:, year-2002:year-2001]
Mkj = Mkj_0212[:, :, year-2002:year-2001].reshape([31, 31])
Pi = Pi_0212[:, :, year-2002:year-2001].reshape([31, 31])
S  = S_0217[:, year-2002:year-2001]
Lp = Lp_0217[:, year-2002:year-2001]
Lh = Lh_0217[:, year-2002:year-2001]
P  = P_0217[:, year-2002:year-2001]
WoP = WoP_0217[:, year-2002:year-2001]

## 以下对应“SolveInitiate.m”文件
I = np.ones([31, 1]) 
X0 = np.zeros([31, 1]) 
X = np.ones([31, 1]) 
R = np.ndarray([31, 1])
G = np.ndarray([31, 1])
V = np.ndarray([31, 1])
Vkmat = np.ndarray([31, 31])
Vjmat = np.ndarray([31, 31])

while ((X0 - X) ** 2).sum() > 1e-20: ## (b)商品市场出清条件，求解总产出
    X0 = X
    R = (1 - T) * Pi @ X0
    I = (1 - Tau) * (1 - alpha) * R
    I = I / I.sum()
    X = (beta * (1 - Tau) + Xi + S / (1 - alpha)) * I / (1 - Tau)
G = (Xi + S / (1 - alpha)) * WoP * N ## (c)政府购买
V = G ** phip * (1 - Tau) ** beta * WoP ** beta * N ** (beta - 1 - phip * chi) * Lh ** (1 - beta) / (1 - beta) ** (1 - beta) ## (d)个体的间接效用
Vkmat = np.tile(V, (1, 31))
Vjmat = np.tile(V.T, (31, 1))
Mukj = (Mkj / np.tile(np.diag(Mkj).reshape(31, 1), (1, 31))) ** (1 / kappa) * Vkmat / Vjmat ## (e)移民成本
Gamkj = ((np.diag(Pi).reshape([31, 1]) @ np.diag(Pi).reshape([1, 31])) / (Pi * Pi.T)) ** (1 / (2 * theta)) ## (f)贸易成本

Paramvalue = [Tau, T, kappa, theta, beta, alpha, phih, phip, chi, sigma, Xi]

In [4]:
# 定义SolveCounter函数，对应“SolveCounter.m”文件
def SolveCounter(XX, ds, dLh, dLp, dGam, dMu, dT, Paramvalue, M, N, N0, S, Mkj, Pi, R, I):
    Tau = Paramvalue[0]
    T = Paramvalue[1]
    kappa = Paramvalue[2]
    theta = Paramvalue[3]
    beta = Paramvalue[4]
    alpha = Paramvalue[5]
    phih = Paramvalue[6]
    phip = Paramvalue[7]
    chi = Paramvalue[8]
    sigma = Paramvalue[9]
    Xi = Paramvalue[10]

    dW = XX[0 : 31]
    dN = XX[31 : 62]
    dP = XX[62 : 93]

    dG = ((1 - alpha) * Xi / S + dS) / ((1 - alpha) * Xi / S + 1) * dW * dN / dP
    dx = (dN / dLp) ** alpha * dW # 单位投入数的价格变动

    inside = dx / dT / (dG ** phip)
    



In [5]:
# 下面开始求解反事实均衡：以2012年为基期，2013-17年的实际情况为反事实，以及其他一些假想的情形为反事实。
dLp = Lp_0217[:, 15:16] / Lp ## 假设土地供给按照2013-17年的情况变化
dLh = Lh_0217[:, 15:16] / Lh
#dLp = np.ones([31, 1])
#dLh = np.ones([31, 1])
dT = np.ones([31, 1]) ## 假设技术水平、转移支付、移民成本和贸易成本不变
dS = np.ones([31, 1])
dGam = np.ones([31, 1])
dMu = np.ones([31, 1])
#dMu = np.loadtxt(r'D:\Python\2022.10长三角城市群\3.量化分析\data\muhat.csv', delimiter=',') ** (-1)

In [6]:
# 导入外生参数
Tau = Paramvalue[0]
T = Paramvalue[1]
kappa = Paramvalue[2]
theta = Paramvalue[3]
beta = Paramvalue[4]
alpha = Paramvalue[5]
phih = Paramvalue[6]
phip = Paramvalue[7]
chi = Paramvalue[8]
sigma = Paramvalue[9]
Xi = Paramvalue[10]


def SolveCounter(XX):
    dW = XX[:31].reshape([31, 1])
    dN = XX[31:62].reshape([31, 1])
    dP = XX[62:].reshape([31, 1])
    dG = ((1 - alpha) * Xi / S + dS) / ((1 - alpha) * Xi /S + 1) * dW * dN / dP

    dx = (dN / dLp) ** alpha * dW

    inside = dx / dT / (dG ** phip)
    dP_cpt = ((Pi.T * (dGam.T ** (-theta))) @ (inside ** (-theta))) ** (-1 / theta)
    inside_tile = np.tile(inside ** (-theta), (1, 31))
    dP_cpt_tile = np.tile((dP_cpt ** theta).T, (31, 1))
    Pi_new = (Pi * (dGam ** (-theta))) * inside_tile * dP_cpt_tile

    dV = dG ** phih * (dW / dP_cpt) ** beta * dN ** (beta - 1 - phih * chi) * dLh ** (1 - beta)
    denominator = (Mkj * dMu ** kappa) @ (dV ** kappa) ## 原代码中没有乘上kappa，但测试了下没啥影响
    dV_tile = np.tile((dV ** kappa).T, (31, 1))
    denominator_tile = np.tile(denominator, (1, 31))
    Mkj_new = (Mkj * dMu ** kappa) * dV_tile / denominator_tile

    N_new = Mkj_new.T @ N0

    R_new = dW * dN * R
    I_new = dW * dN * I
    X_new = np.linalg.inv(Pi_new) @ (R_new / (1 - T))

    res1 = dP - dP_cpt
    res2 = dN - N_new / N
    res3 = X_new - (beta * (1 - Tau) + Xi + S * dS / (1 - alpha)) * I / Tau # 此处代码有误，应该是I_new，但是修改后就求不出解了

    return np.concatenate([res1, res2, res3]).flatten()



In [7]:
XX = root(SolveCounter, x0=np.ones(3*31), method='lm')
dW = XX.x[:31].reshape([31, 1])
dN = XX.x[31:62].reshape([31, 1])
dP = XX.x[62:].reshape([31, 1])
# dW / dP # 如果单独查看dW和dP，结果会非常奇怪

In [8]:
dG = ((1 - alpha) * Xi / S + dS) / ((1 - alpha) * Xi /S + 1) * dW * dN / dP

dx = (dN / dLp) ** alpha * dW

inside = dx / dT / (dG ** phip)
dP_cpt = ((Pi.T * (dGam.T ** (-theta))) @ (inside ** (-theta))) ** (-1 / theta)
inside_tile = np.tile(inside ** (-theta), (1, 31))
dP_cpt_tile = np.tile((dP_cpt ** theta).T, (31, 1))
Pi_new = (Pi * (dGam ** (-theta))) * inside_tile * dP_cpt_tile

dV = dG ** phih * (dW / dP_cpt) ** beta * dN ** (beta - 1 - phih * chi) * dLh ** (1 - beta)
denominator = (Mkj * dMu ** kappa) @ (dV ** kappa) ## 原代码中没有乘上kappa，但测试了下没啥影响
dV_tile = np.tile((dV ** kappa).T, (31, 1))
denominator_tile = np.tile(denominator, (1, 31))
Mkj_new = (Mkj * dMu ** kappa) * dV_tile / denominator_tile

N_new = Mkj_new.T @ N0

R_new = dW * dN * R
I_new = dW * dN * I
X_new = np.linalg.inv(Pi_new) @ (R_new / (1 - T))

dV

array([[1.01420539],
       [1.01726964],
       [1.02353281],
       [1.02152026],
       [1.02790599],
       [1.01553919],
       [1.01639179],
       [1.01275367],
       [1.01444332],
       [1.01891741],
       [1.02989761],
       [1.01775123],
       [1.03414532],
       [1.03015609],
       [1.02034593],
       [1.02438949],
       [1.03000879],
       [1.02388441],
       [1.03062835],
       [1.02824619],
       [1.0234833 ],
       [1.03150374],
       [1.03056802],
       [1.05582646],
       [1.04357471],
       [1.02928771],
       [1.03316965],
       [1.04079761],
       [1.03650743],
       [1.04395053],
       [1.00033926]])